# Zad 1

We wszystkich zadaniach w raporcie po wykresach trzeba dać komentarz. Komentarz ma mówić który estymator jest dobry a który nie i dlaczego.

In [90]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_SEED = 12345
np.random.seed(RANDOM_SEED)

import os
zad1_dir = "zad1"
os.makedirs(zad1_dir, exist_ok=True)
zad2_dir = "zad2"
os.makedirs(zad2_dir, exist_ok=True)
zad3_dir = "zad3"
os.makedirs(zad3_dir, exist_ok=True)
zad4_dir = "zad4"
os.makedirs(zad4_dir, exist_ok=True)

In [91]:
# pomocnicza funkcja do budowania wag dla Theta_3

def generate_weights(n):
    # zwraca wektor wag dla Theta_3
    weights = np.random.rand(n)
    weights /= np.sum(weights)  # normalizacja wag, aby suma była równa 1
    return weights

In [93]:
# funkcje pomocnicze

def generate_sample_normal(theta, sigma, n):
    return np.random.normal(loc=theta, scale=sigma, size=n)

def theta4_estimator(sample):
  n = len(sample)
  X_sorted = np.sort(sample)

  i = np.arange(1, n + 1)
  w = norm.pdf(norm.ppf((i - 1) / n)) - norm.pdf(norm.ppf(i / n))

  theta4 = np.sum(X_sorted * w)
  return theta4

def trimmed_mean(sample, alpha):
  n = len(sample)
  sorted_sample = np.sort(sample)
  trimmed_sample = sorted_sample[int(alpha * n):int((1 - alpha) * n)]
  return np.mean(trimmed_sample)


def get_statistics_from_sample(sample, weights):
    return {
        'Theta_1': np.mean(sample),
        'Theta_2': np.median(sample),
        'Theta_3': np.sum(sample * weights),
        'Theta_4': theta4_estimator(sample),
        'Theta_5': len(sample) / np.sum(1 / sample),
        'Theta_6': np.cbrt((np.mean(sample**3))),
        'Theta_7': trimmed_mean(sample, 0.1)
    }

def get_all_statistics(repetitions, theta, sigma, n, weights):
    stats_list = []
    for _ in range(repetitions):
        samp = generate_sample_normal(theta, sigma, n) # (X1, X2, ..., Xn)
        stats = get_statistics_from_sample(samp, weights) # dict {'stat1': val1, 'stat2': val2, ...}
        stats_list.append(stats) # list of dicts of length {repetitions} with stats

    df_stats = pd.DataFrame(stats_list)
    df_stats = df_stats[['Theta_1','Theta_2','Theta_3', 'Theta_4', 'Theta_5', 'Theta_6', 'Theta_7']]
    return df_stats


In [ ]:
def create_boxplot(df_melted, theta, sigma, n, with_theta_5=True):
  subset_df = df_melted[(df_melted['theta'] == theta) & (df_melted['sigma'] == sigma)]
  plt.figure(figsize=(12, 8))
  sns.boxplot(x='Estimator', y='Value', data=subset_df)
  plt.title(f'Box plots of Estimators for θ={theta}, σ={sigma}, n={n}')
  plt.ylabel('Estimator Value')
  plt.xlabel('Estimator')
  plt.grid(True)
  # plt.show()
  plot_filename = os.path.join(zad1_dir, f"theta{theta}_sigma{sigma}_n{n}_theta5{with_theta_5}.png")
  plt.savefig(plot_filename, bbox_inches='tight')
  plt.close()


In [95]:
def calculate_estimator_metrics(estimator_values, true_parameter, estimator_name = ""):
    variance = np.var(estimator_values)
    bias = np.mean(estimator_values) - true_parameter
    mse = np.mean((estimator_values - true_parameter)**2) # Equivalent to variance + bias^2

    return {'estimator': estimator_name, 'variance': variance, 'bias': bias, 'mse': mse}

In [99]:
# parametry
repetitions = 10000
params = [(0,1),(0,2),(4,1),(4,2)]  # lista (theta, sigma)

def run_experiments(n):
  weights = generate_weights(n) # jeden wektor wag do wszystkich powtórzeń

  all_stats_data = []
  for theta, sigma in params:
      print(f"Running experiment for theta={theta}, sigma={sigma}")
      df_stats = get_all_statistics(repetitions, theta, sigma, n, weights)
      df_stats['theta'] = theta
      df_stats['sigma'] = sigma
      all_stats_data.append(df_stats)

  combined_df_stats = pd.concat(all_stats_data)

  # combined_df_stats.head()
  # Reshape for plotting
  df_melted = combined_df_stats.melt(id_vars=['theta', 'sigma'],
                                  var_name='Estimator',
                                  value_name='Value')

  # Create box plots with theta_5
  for theta, sigma in params:
      create_boxplot(df_melted, theta, sigma, n, with_theta_5=True)

  # Create box plots without theta_5
  df_melted_no_theta_5 = df_melted[df_melted['Estimator'] != 'Theta_5']

  pd.set_option('display.float_format', lambda x: f'{x:,.6f}')

  for theta, sigma in params:
      create_boxplot(df_melted_no_theta_5, theta, sigma, n, with_theta_5=False)
      metrics_list = []
      for estimator in ['Theta_1', 'Theta_2', 'Theta_3', 'Theta_4', 'Theta_5', 'Theta_6', 'Theta_7']:
        subset_df = df_melted[(df_melted['theta'] == theta) & (df_melted['sigma'] == sigma) & (df_melted['Estimator'] == estimator)]
        metrics = calculate_estimator_metrics(subset_df['Value'], theta, estimator_name=estimator)
        metrics_list.append(metrics)
      df_metrics = pd.DataFrame(metrics_list)

      md_filename = os.path.join(zad1_dir, f"summary_theta{theta}_sigma{sigma}_n{n}.md")
      with open(md_filename, "w") as f:
          f.write(f"**Podsumowanie estymatorów dla θ={theta}, σ={sigma}, n={n}** \n\n")
          f.write(df_metrics.to_markdown(index=False))
          f.write("\n")

      print(f"\n📊 Podsumowanie estymatorów dla θ={theta}, σ={sigma}, n={n}\n")
      print(df_metrics.to_string(index=False))
      print("\n" + "-"*80 + "\n")

In [100]:
for n in [20, 50, 100]:
  run_experiments(n)

Running experiment for theta=0, sigma=1
Running experiment for theta=0, sigma=2
Running experiment for theta=4, sigma=1
Running experiment for theta=4, sigma=2

📊 Podsumowanie estymatorów dla θ=0, σ=1, n=20

estimator      variance      bias           mse
  Theta_1      0.051745  0.000518      0.051745
  Theta_2      0.074011  0.000094      0.074011
  Theta_3      0.072403  0.001054      0.072404
  Theta_4      0.023174  0.932169      0.892113
  Theta_5 19,919.214642  1.382095 19,921.124828
  Theta_6      0.688186 -0.001435      0.688188
  Theta_7      0.054298  0.001218      0.054299

--------------------------------------------------------------------------------


📊 Podsumowanie estymatorów dla θ=0, σ=2, n=20

estimator     variance      bias          mse
  Theta_1     0.196514  0.000282     0.196514
  Theta_2     0.290236 -0.002669     0.290243
  Theta_3     0.280163 -0.001622     0.280165
  Theta_4     0.093146  1.859066     3.549274
  Theta_5 9,515.319995  1.357458 9,517.162688
 

# Zad 2

In [42]:
import scipy.stats

n = 50
repetitions = 10000
params = [(0,1),(0,2),(4,1),(4,2)]

In [43]:
def generate_sample_normal(theta, sigma, n):
    return np.random.normal(loc=theta, scale=sigma, size=n)

def generate_sample_logist(theta, sigma, n):
    return np.random.logistic(loc=theta, scale=sigma, size=n)

def generate_sample_cauchy(theta, sigma, n):
    # return np.random.standard_cauchy(n) * sigma + theta
    return scipy.stats.cauchy(loc=theta, scale=sigma).rvs(size=n)

def get_statistics_from_sample(sample):
    return {
        'Theta_1': np.mean(sample),
        'Theta_2': np.median(sample)
    }

def get_all_statistics(repetitions, theta, sigma, n, sample_generation_function):
    stats_list = []
    for _ in range(repetitions):
        samp = sample_generation_function(theta, sigma, n) # (X1, X2, ..., Xn)
        stats = get_statistics_from_sample(samp) # dict {'stat1': val1, 'stat2': val2, ...}
        stats_list.append(stats) # list of dicts of length {repetitions} with stats

    df_stats = pd.DataFrame(stats_list)
    df_stats = df_stats[['Theta_1','Theta_2']]
    return df_stats

In [44]:
def calculate_for_distribution(distribution_name):
  if distribution_name == 'normal':
    sample_generation_function = generate_sample_normal
  elif distribution_name == 'logist':
    sample_generation_function = generate_sample_logist
  elif distribution_name == 'cauchy':
    sample_generation_function = generate_sample_cauchy

  all_stats_data = []

  for theta, sigma in params:
      # print(f"Running experiment for {distribution_name} distribution for theta={theta}, sigma={sigma}")
      df_stats = get_all_statistics(repetitions, theta, sigma, n, sample_generation_function)
      df_stats['theta'] = theta
      df_stats['sigma'] = sigma
      df_stats['Distribution'] = distribution_name
      all_stats_data.append(df_stats)

  return pd.concat(all_stats_data)

In [46]:
df_normal = calculate_for_distribution('normal')
df_logist = calculate_for_distribution('logist')
df_cauchy = calculate_for_distribution('cauchy')

df_all = pd.concat([df_normal, df_logist, df_cauchy])

df_melted = df_all.melt(id_vars=['theta', 'sigma', 'Distribution'],
                        var_name='Estimator',
                        value_name='Value')

df_melted['EstimatorLabel'] = df_melted['Distribution'] + "_" + df_melted['Estimator']

pd.set_option('display.float_format', lambda x: f'{x:,.6f}')

for theta, sigma in params:
    subset_df = df_melted[(df_melted['theta'] == theta) & (df_melted['sigma'] == sigma)]

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='EstimatorLabel', y='Value', data=subset_df)
    plt.title(f'Estymatory θ dla różnych rozkładów (θ={theta}, σ={sigma})')
    plt.ylabel('Wartość estymatora')
    plt.xlabel('Rozkład i estymator')
    plt.grid(True)
    # Eksport wykresu do podfolderu zad2
    plot_filename = os.path.join(zad2_dir, f"boxplot_theta{theta}_sigma{sigma}.png")
    plt.savefig(plot_filename, bbox_inches='tight')
    plt.close()

    summary_rows = []
    for (dist, est), group in subset_df.groupby(['Distribution', 'Estimator']):
        values = group['Value']
        est_mean = np.mean(values)
        bias = est_mean - theta
        variance = np.var(values)
        mse = variance + bias**2
        summary_rows.append({
            'Distribution': dist,
            'Estimator': est,
            'Mean': est_mean,
            'Bias': bias,
            'Variance': variance,
            'MSE': mse
        })

    df_summary = pd.DataFrame(summary_rows)
    df_summary = df_summary[['Distribution', 'Estimator', 'Bias', 'Variance', 'MSE']]

    print(f"\n📊 Podsumowanie dla θ={theta}, σ={sigma}\n")
    print(df_summary.to_string(index=False))
    print("\n" + "-"*80 + "\n")
    # Eksport dataframe do pliku markdown w podfolderze zad2
    md_filename = os.path.join(zad2_dir, f"summary_theta{theta}_sigma{sigma}.md")
    with open(md_filename, "w") as f:
        f.write(f"**Podsumowanie estymatorów dla θ={theta}, σ={sigma}\n\n**")
        f.write(df_summary.to_markdown(index=False))
        f.write("\n")

# plot without cauchy_theta_1
for theta, sigma in params:
    subset_df = df_melted[(df_melted['theta'] == theta) & (df_melted['sigma'] == sigma) & (df_melted['EstimatorLabel'] != 'cauchy_Theta_1')]

    plt.figure(figsize=(10, 6))
    sns.boxplot(x='EstimatorLabel', y='Value', data=subset_df)
    plt.title(f'Estymatory θ dla różnych rozkładów (θ={theta}, σ={sigma})')
    plt.ylabel('Wartość estymatora')
    plt.xlabel('Rozkład i estymator')
    plt.grid(True)
    # Eksport wykresu bez cauchy_theta_1 do podfolderu zad2
    plot_filename = os.path.join(zad2_dir, f"boxplot_theta{theta}_sigma{sigma}_no_cauchy_theta1.png")
    plt.savefig(plot_filename, bbox_inches='tight')
    plt.close()


📊 Podsumowanie dla θ=0, σ=1

Distribution Estimator      Bias      Variance           MSE
      cauchy   Theta_1  1.012926 12,563.651481 12,564.677500
      cauchy   Theta_2 -0.003036      0.051657      0.051666
      logist   Theta_1 -0.002363      0.066789      0.066795
      logist   Theta_2 -0.001398      0.078830      0.078832
      normal   Theta_1 -0.000236      0.021068      0.021069
      normal   Theta_2 -0.000555      0.031859      0.031859

--------------------------------------------------------------------------------


📊 Podsumowanie dla θ=0, σ=2

Distribution Estimator      Bias       Variance            MSE
      cauchy   Theta_1 -6.282730 269,832.954730 269,872.427433
      cauchy   Theta_2 -0.008540       0.203015       0.203088
      logist   Theta_1 -0.001405       0.265433       0.265435
      logist   Theta_2 -0.001117       0.319740       0.319741
      normal   Theta_1  0.003956       0.079864       0.079879
      normal   Theta_2  0.004033       0.122367     

# Zad 3

generujemy o jedną obserwację losową mniej (n-1). dorzucamy jedną dużą obserwację k na koniec.

Estymator:
- mediana powinna być odporna na taką jedną za dużą obserwację
- średnia nie jest tak dobra

Wykres:
np. liniowy y - bias. x - różne wartości k

1. najpierw wylosuj n-1 liczb
2. dla nich sprawdzaj różne k


In [65]:
k_values = range(10, 101, 10)
repetitions = 10_000

def get_statistics_from_sample(sample):
  return {
    'Theta_1': np.mean(sample),
    'Theta_2': np.median(sample)
  }

def get_experiments(repetitions):
  stats_list = []
  for _ in range(repetitions):
    sample = generate_sample_normal(0, 1, 49)
    for k in k_values:
      sample_k = np.append(sample, k)
      stats = get_statistics_from_sample(sample_k)
      stats['k'] = k
      stats_list.append(stats)

  df_stats = pd.DataFrame(stats_list)
  return df_stats


df_stats = get_experiments(repetitions)

estimator_metrics = {}
for k in k_values:
  subset_df = df_stats[df_stats['k'] == k]
  theta_1_metrics = calculate_estimator_metrics(subset_df['Theta_1'], 0)
  theta_2_metrics = calculate_estimator_metrics(subset_df['Theta_2'], 0)
  estimator_metrics[k] = {
    'Theta_1': theta_1_metrics,
    'Theta_2': theta_2_metrics
  }

rows = []
for k, estimators in estimator_metrics.items():
    for theta_name, metrics in estimators.items():
        rows.append({
            'k': k,
            'Estimator': theta_name,
            'Bias': metrics['bias'],
            'Variance': metrics['variance'],
            'MSE': metrics['mse']
        })

df_metrics = pd.DataFrame(rows)

md_filename = os.path.join(zad3_dir, "summary_zad3.md")
with open(md_filename, "w") as f:
    f.write("**Podsumowanie estymatorów dla różnych wartości k**\n\n")
    f.write(df_metrics.to_markdown(index=False))
    f.write("\n")

df_metrics.head(20)

,k,Estimator,Bias,Variance,MSE
0,10,Theta_1,0.200490,0.019556,0.059752
1,10,Theta_2,0.023911,0.031864,0.032435
2,20,Theta_1,0.400490,0.019556,0.179948
3,20,Theta_2,0.023911,0.031864,0.032435
4,30,Theta_1,0.600490,0.019556,0.380144
5,30,Theta_2,0.023911,0.031864,0.032435
6,40,Theta_1,0.800490,0.019556,0.660340
7,40,Theta_2,0.023911,0.031864,0.032435
8,50,Theta_1,1.000490,0.019556,1.020537
9,50,Theta_2,0.023911,0.031864,0.032435


In [59]:
import seaborn as sns

sns.lineplot(data=df_metrics, x='k', y='Bias', hue='Estimator')
plt.title('Bias wzglem k')
plt.xlabel('k')
plt.ylabel('Bias')
# plt.show()
plot_filename = os.path.join(zad3_dir, "bias_vs_k.png")
plt.savefig(plot_filename, bbox_inches='tight')
plt.close()

sns.lineplot(data=df_metrics, x='k', y='Variance', hue='Estimator')
plt.title('Variance względem k')
plt.xlabel('k')
plt.ylabel('Variance')
# plt.show()
plot_filename = os.path.join(zad3_dir, "variance_vs_k.png")
plt.savefig(plot_filename, bbox_inches='tight')
plt.close()

sns.lineplot(data=df_metrics, x='k', y='MSE', hue='Estimator')
plt.title('MSE względem k')
plt.xlabel('k')
plt.ylabel('MSE')
# plt.show()
plot_filename = os.path.join(zad3_dir, "mse_vs_k.png")
plt.savefig(plot_filename, bbox_inches='tight')
plt.close()

# Zad 4

In [78]:
n = 50
repetitions = 10_000
params = [(0,1),(0,4),(4,1),(4,4)] # mu, Theta

In [80]:
def generate_sample_normal_v4(mu, theta, n):
    return np.random.normal(loc=mu, scale=np.sqrt(theta), size=n)


def get_statistics_from_sample_v4(sample):
    return {
        'Theta_1': np.var(sample, ddof=1),
        'Theta_2': np.sum((sample - np.median(sample))**2) / (len(sample) - 1),
        'Theta_3': np.median((sample - np.mean(sample))**2)
    }

In [81]:
def get_estimator_stats(repetitions, mu, theta, n):
  stats_list = []
  for _ in range(repetitions):
    sample = generate_sample_normal_v4(mu, theta, n)
    stats = get_statistics_from_sample_v4(sample)
    stats_list.append(stats)

  df_stats = pd.DataFrame(stats_list)
  df_stats = df_stats[['Theta_1','Theta_2','Theta_3']]
  return df_stats





In [85]:
def plot_estimator_stats(df_stats, mu, theta):
  df_melted = df_stats.melt(var_name="Estimator", value_name="Value")
  sns.boxplot(x="Estimator", y="Value", data=df_melted)
  plt.title(f"Box plots of Estimators for μ={mu}, θ={theta}")
  plt.ylabel("Estimator Value")
  plt.xlabel("Estimator")
  plt.grid(True)
  # plt.show()
  plot_filename = os.path.join(zad4_dir, f"mu{mu}_theta{theta}.png")
  plt.savefig(plot_filename, bbox_inches='tight')
  plt.close()


In [88]:
def get_combined_experiment_results(repetitions, n, mu, theta):
  df_stats = get_estimator_stats(repetitions, mu, theta, n)
  plot_estimator_stats(df_stats, mu, theta)
  metrics_list = []
  metrics_list.append(calculate_estimator_metrics(df_stats['Theta_1'], theta, estimator_name='Theta_1'))
  metrics_list.append(calculate_estimator_metrics(df_stats['Theta_2'], theta, estimator_name='Theta_2'))
  metrics_list.append(calculate_estimator_metrics(df_stats['Theta_3'], theta, estimator_name='Theta_3'))
  df_metrics = pd.DataFrame(metrics_list)

  md_filename = os.path.join(zad4_dir, f"summary_zad4_mu{mu}_theta{theta}.md")
  with open(md_filename, "w") as f:
      f.write(f"**Podsumowanie estymatorów μ={mu}, θ={theta}** \n\n")
      f.write(df_metrics.to_markdown(index=False))
      f.write("\n")
  print(df_metrics)

In [89]:
for mu, theta in params:
  get_combined_experiment_results(repetitions, n, mu, theta)

  estimator  variance      bias      mse
0   Theta_1  0.041318  0.000894 0.041319
1   Theta_2  0.042428  0.011540 0.042561
2   Theta_3  0.022596 -0.535584 0.309446
  estimator  variance      bias      mse
0   Theta_1  0.655936 -0.004036 0.655952
1   Theta_2  0.675235  0.040251 0.676855
2   Theta_3  0.367530 -2.144185 4.965059
  estimator  variance      bias      mse
0   Theta_1  0.041464 -0.002162 0.041469
1   Theta_2  0.042640  0.008771 0.042717
2   Theta_3  0.022632 -0.536303 0.310252
  estimator  variance      bias      mse
0   Theta_1  0.651435  0.005369 0.651464
1   Theta_2  0.670910  0.048157 0.673229
2   Theta_3  0.361556 -2.144992 4.962549


# Raport
- Treść zadania
- Wyniki
  - wykresy podpisane dobrze
  - jak np. w pierwszym zadaniu masz pary wykres, tabelka dla danych parametrów rozkładu normalnego, to najlepiej trzymać je obok siebie w raporcie. a nie że najpierw wykresy, potem tabelki
- Przeanalizowane wyniki - komentarze
  - Wnioski o porównywaniu estymatorów. że ten i ten taki sam, któryśtam mega słaby bo cośtam itd itd.
  - Porównanie estymatorów dla różnych parametrów rozkładu normalnego
  - Porównanie estymatorów dla różnych rozmiarów sample
  - Wpływ obserwacji odstających
  - Komentarze można na końcu zadań albo w trakcie, jak chcesz
  - Komentarz jaki estymator wybrałeś w pierwszym zadaniu gdzie trzeba jakiś swój dać musi być. Dlaczego taki i jak działa.

- Kod można osobno dosłać